In [1]:
pip install stepic

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import os
import binascii
from collections import deque #library used for rotation 
import matplotlib.pyplot as plt
import stepic
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from PIL import ImageTk, Image

In [3]:
# all required matrix

RS_matrix=[[0x01,0xA4,0x55,0x87,0x5A,0x58,0xDB,0x9E],[0xA4,0x56,0x82,0xF3,0x1E,0xC6,0x68,0xE5],[0x02,0xA1,0xFC,0xC1,0x47,0xAE,0x3D,0x19],[0xA4,0x55,0x87,0x5A,0x58,0xDB,0x9E,0x03]]
tq0=[[0x8,0x1,0x7,0xD,0x6,0xF,0x3,0x2,0x0,0xB,0x5,0x9,0xE,0xC,0xA,0x4],[0xE,0xC,0xB,0x8,0x1,0x2,0x3,0x5,0xF,0x4,0xA,0x6,0x7,0x0,0x9,0xD],[0xB,0xA,0x5,0xE,0x6,0xD,0x9,0x0,0xC,0x8,0xF,0x3,0x2,0x4,0x7,0x1],[0xD,0x7,0xF,0x4,0x1,0x2,0x6,0xE,0x9,0xB,0x3,0x0,0x8,0x5,0xC,0xA]]
tq1=[[0x2,0x8,0xB,0xD,0xF,0x7,0x6,0xE,0x3,0x1,0x9,0x4,0x0,0xA,0xC,0x5],[0x1,0xE,0x2,0xB,0x4,0xC,0x3,0x7,0x6,0xD,0xA,0x5,0xF,0x9,0x0,0x8],[0x4,0xC,0x7,0x5,0x1,0x6,0x9,0xA,0x0,0xE,0xD,0x8,0x2,0xB,0x3,0xF],[0xB,0x9,0x5,0x1,0xC,0x3,0xD,0xE,0x6,0x4,0x7,0xF,0x2,0x0,0x8,0xA]]

MDS=[[0x01,0xEF,0x5B,0x5B],[0x5B,0xEF,0xEF,0x01],[0xEF,0x5B,0x01,0xEF],[0xEF,0x01,0xEF,0x5B]]

primes=[11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997, 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 1213, 1217, 1223, 1229, 1231, 1237, 1249, 1259, 1277, 1279, 1283, 1289, 1291, 1297, 1301, 1303, 1307, 1319, 1321, 1327, 1361, 1367, 1373, 1381, 1399, 1409, 1423, 1427, 1429, 1433, 1439, 1447, 1451, 1453, 1459, 1471, 1481, 1483, 1487, 1489, 1493, 1499, 1511, 1523, 1531, 1543, 1549, 1553, 1559, 1567, 1571, 1579, 1583, 1597, 1601, 1607, 1609, 1613, 1619, 1621, 1627, 1637, 1657, 1663, 1667, 1669, 1693, 1697, 1699, 1709, 1721, 1723, 1733, 1741, 1747, 1753, 1759, 1777, 1783, 1787, 1789, 1801, 1811, 1823, 1831, 1847, 1861, 1867, 1871, 1873, 1877, 1879, 1889, 1901, 1907, 1913, 1931, 1933, 1949, 1951, 1973, 1979, 1987, 1993, 1997, 1999]

# Polynomials used for multiplication in RF and MDS matrix 
gf_mod = 2**8 + 2**6 + 2**5 + 2**3 + 1
rs_mod = 2**8 + 2**6 + 2**3 + 2**2 + 1

t=[tq0,tq1]
S0=[]
S1=[]

In [4]:
# Function to multiply in Galois Field with correct polynomial
# Here modulus is used as the value of 2^n in the polynomial
def gf2n_multiply(a, b,modulus):
    overflow = 0x100
    sum1 = 0
    while (b > 0):
        if (b & 1):
            sum1 = sum1 ^ a
        b = b >> 1
        a = a << 1
        if (a & overflow):
            a = a ^ modulus
    return sum1


In [5]:
# Right rotation of number with rotation and bits as the parameter

def ROR(num,rot,bits):
    num=bin(num)[2:]
    num=num.zfill(bits)
    num=[int(i) for i in num]

    items=deque(num)
    items.rotate(rot)
    num=list(items)
    num=''.join([str(i) for i in num])
    num=int(num,2)
    return num


In [6]:
# Left rotation of number with rotation and bits as the parameter

def ROL(num,rot,bits):
    num=bin(num)[2:]
    num=num.zfill(bits)
    num=[int(i) for i in num]

    items=deque(num)
    items.rotate(-rot)
    num=list(items)
    num=''.join([str(i) for i in num])
    num=int(num,2)
    return num


In [7]:
# Permuatation function q1 used in the SBOX
def q1(inp):
    
    t0=t[1][0]
    t1=t[1][1]
    t2=t[1][2]
    t3=t[1][3]

    inp=bin(inp)[2:]
    inp=inp.zfill(8)
    a0=int(inp[:4],2)
    b0=int(inp[4:],2)
    a1=a0^b0
    b1=a0^(ROR(b0,1,4))^((8*a0)%16)
    a2=t0[a1]
    b2=t1[b1]
    a3=a2^b2
    b3=a2^(ROR(b2,1,4))^((8*a2)%16)
    a4=t2[a3]
    b4=t3[b3]
    y=16*b4+a4
    return y


In [8]:
# Permuatation function q0 used in the SBOX

def q0(inp):

    t0=t[0][0]
    t1=t[0][1]
    t2=t[0][2]
    t3=t[0][3]

    inp=bin(inp)[2:]
    inp=inp.zfill(8)
    a0=int(inp[:4],2)
    b0=int(inp[4:],2)
    a1=a0^b0
    b1=a0^(ROR(b0,1,4))^((8*a0)%16)
    a2=t0[a1]
    b2=t1[b1]
    a3=a2^b2
    b3=a2^(ROR(b2,1,4))^((8*a2)%16)
    a4=t2[a3]
    b4=t3[b3]
    y=16*b4+a4
    return y


In [9]:
# pseudo-Hadamard transform (PHT) function 

def PHT(a,b):
    num1=(a+b)%(pow(2,32))
    num2=(a+2*b)%pow(2,32)
    return num1,num2

In [10]:
# g function used inside the F function

def g_function(inp_r):

    global S0,S1

    S_0=S0
    S_1=S1
    arr=[]
    h=hex(inp_r)[2:].zfill(8)

    for i in range(0,len(h),2):
        tmp=int(h[i:i+2],16)
        arr.append(tmp)
    arr=arr[::-1]

    inp0=arr[0]
    inp1=arr[1]
    inp2=arr[2]
    inp3=arr[3]

    output=[0,0,0,0]

    # Taking the output of the SBOXES used in the G_function
    output[0] = q1(q0(q0(inp0) ^ S_0[0]) ^ S_1[0])
    output[1] = q0(q0(q1(inp1) ^ S_0[1]) ^ S_1[1])
    output[2] = q1(q1(q0(inp2) ^ S_0[2]) ^ S_1[2])
    output[3] = q0(q1(q1(inp3) ^ S_0[3]) ^ S_1[3])

    # Matrix multiplication under the Galois filed with modulus of GF
    output=mat_mul(MDS,output,gf_mod)
    # Little endian
    output=output[::-1]

    # Combining 4 8-bit numbers to 1 32-bit number
    output=int(''.join([bin(i)[2:].zfill(8) for i in output]),2)

    return output

In [11]:
# A helper function for the main function H used in round key generation

def helper_h(inp1,M1,M2):

    output=[0,0,0,0]

    output[0] = q1(q0(q0(inp1) ^ M1[0]) ^ M2[0])
    output[1] = q0(q0(q1(inp1) ^ M1[1]) ^ M2[1])
    output[2] = q1(q1(q0(inp1) ^ M1[2]) ^ M2[2])
    output[3] = q0(q1(q1(inp1) ^ M1[3]) ^ M2[3])
    output=mat_mul(MDS,output,gf_mod)

    return output


In [12]:
# H function used in key scheduling
def h_function(M_even,M_odd):

    M0=M_even[0]
    M2=M_even[1]
    M1=M_odd[0]
    M3=M_odd[1]

    K_keys=[]

    # Loop for making 40 keys
    for i in range(0,40,2):
        inp1=i
        inp2=i+1

        # Calling helper function which is performing the S-Box operations

        key1=helper_h(inp1,M2,M0)
        key2=helper_h(inp2,M3,M1)

        fin_key1=[]
        fin_key2=[]

        # Making the 4 8-bit keys to a combined 32 bit key with adjusting little endian 
        for i in range(4):
            fin_key1.append(bin(key1[i])[2:].zfill(8))
            fin_key2.append(bin(key2[i])[2:].zfill(8))

        fin_key1=fin_key1[::-1]
        fin_key2=fin_key2[::-1]

        # binary to decimal conversion
        key1=int(''.join(fin_key1),2)
        key2=int(''.join(fin_key2),2)

        # Rotating the key by 8 bits
        key2=ROL(key2,8,32)

        # pseudo-Hadamard transform of the key1 and key2
        key1,key2=PHT(key1,key2)

        # Left rotation by 9 bits of key2
        key2=ROL(key2,9,32)

        # Finally appending the keys to main key list
        K_keys.append(key1)
        K_keys.append(key2)

    return K_keys

In [13]:
# A function for matrix multiplication which uses the Field multiplication and addition rules

def mat_mul(mat1,mat2,modulus):
    row1=len(mat1)
    col1=len(mat1[0])

    fin=[]
    for i in range(row1):
        val=0
        for j in range(col1):
            tmp1=gf2n_multiply((mat1[i][j]),mat2[j],modulus)
            val=val^tmp1
        fin.append(val)
    return fin

In [14]:
# Main function for Key scheduling 
def key_schedule(key):

    global S0,S1
    m_array=[] 

    # array of 16 8 bit-keys provided by user
    for i in range(0,len(key),2):
        tmp=int(key[i:i+2],16)
        m_array.append(tmp)

    # Making the Sbox S0 and S1 with RS modulo multiplication

    S0=mat_mul(RS_matrix,m_array[:8],rs_mod)
    S1=mat_mul(RS_matrix,m_array[8:16],rs_mod)

    # Odd even matrix for round keys generation
    M_even=[]
    M_odd=[]

    val=0

    # Making the even and odd lists
    for i in range(0,len(m_array),4):
        tmp=m_array[i:i+4]
        if(val%2==0):
            M_even.append(tmp)
        else:
            M_odd.append(tmp)
        val+=1
    # Calling H function with parameter Meven and Modd

    K_keys=h_function(M_even,M_odd)

    # for i in range(0,40,2):
    #     print(hex(K_keys[i])[2:].zfill(8),hex(K_keys[i+1])[2:].zfill(8))

    return K_keys

In [15]:
# Function for Input Whitening

def whitening(plaintext,white_keys):
    plain=[]
    new_key=[]
    val=0
    # Converting plaintext to a array of 16 length

    for i in range(0,len(plaintext),2):
        tmp=int(plaintext[i:i+2],16)
        plain.append(tmp)
    arr2=[]

    # taking 4 8-bit number together and then adjusting little endian
    for i in range(0,len(plain),4):
        tmp=plain[i:i+4]
        tmp=tmp[::-1]   #reversing the list for little endian adjustments
        arr2+=tmp
    plain=arr2

    # Expanding 4 32 bit numbers to 16 8-bit number array
    for j in range(len(white_keys)):
        x=hex(white_keys[j])[2:].zfill(8)
        for k in range(0,len(x),2):
            tmp=int(x[k:k+2],16)
            new_key.append(tmp)
    r_array=[]

    # Now both key and plaintext is 16 8-bit array so we can XOR
    for i in range(len(plain)):
        r_array.append(new_key[i]^plain[i])

    # Returning the round State

    r0=r_array[:4]
    r1=r_array[4:8]
    r2=r_array[8:12]
    r3=r_array[12:16]
    r_array=[r0,r1,r2,r3]


    return r_array

In [16]:
# The F function used in Encryption

def f_function(r_array,k1,k2):

    r0=r_array[0]
    r1=r_array[1]

    # Rotationg left
    r1=ROL(r1,8,32) 
       # Calling G function for  r0 and r1 and then obtaining t0 and t1
    t0=g_function(r0)
    t1=g_function(r1)

    # print(hex(t0),end= " ")
    # print(hex(t1))
    # exit()

    # pseudo-Hadamard transform of t0 and t1
    t0,t1=PHT(t0,t1)

    # addition of round keys with modulo 2^32 
    f0=(t0+k1)%pow(2,32)
    f1=(t1+k2)%pow(2,32)

    # returning f0 and f1 
    return f0,f1


In [17]:
# Decryption fucntion
def decrypt(ciphertext,key):

    # Making the required keys with scheduling
    round_keys=key_schedule(key)
    white_keys=round_keys[:4]
    output_keys=round_keys[4:8]

    # Converting ciphertext to array of 16
    ciphertext=[ciphertext[i:i+8] for i in range(0,len(ciphertext),8) ]
    r_array=[]

    # Adjusting the little endian format
    for i in ciphertext:
        q=i
        s=[]
        for j in range(0,len(q),2):
            s.append(q[j:j+2])
        s=s[::-1]
        s=''.join(s)
        r_array.append(int(s,16))

    # Ciphertext whitening with output whiten keys
    for j in range(len(output_keys)):
        r_array[j]=r_array[j]^output_keys[j]

    # Doing the criss cross swapping in Fiestal cipher
    r_array=[r_array[2],r_array[3],r_array[0],r_array[1]]

    # Calling the loop for 16 rounds
    for r in range(15,-1,-1):

        # Reversing the states ,the 3rd and 4th element will be 1st and 2nd element of previous round state array
        a=r_array[2]
        b=r_array[3]
        c2=r_array[0]
        c3=r_array[1]

        # Calling the F function with the 3rd and 4th element
        f0,f1=f_function([a,b],round_keys[2*r+8],round_keys[2*r+9])
        
        # Reversing to get the r2 and r3 of previous round in ecryption
        r2=ROL(c2,1,32)
        r2=r2^f0

        r3=f1^c3
        r3=ROR(r3,1,32)
        
        r_array=[a,b,r2,r3]

    # After 16 rounds ,whitening the array with input whiten keys this time

    for i in range(4):
        r_array[i]=hex(r_array[i]^white_keys[i])[2:].zfill(8)

    ans=""

    # Printing the output in Big Endian format
    for i in r_array:
        tmp=[]
        for j in range(0,len(i),2):
            tmp.append(i[j:j+2])
        tmp=tmp[::-1]
        ans+=''.join(tmp)
    ans=ans.lstrip('0')
    ans=binascii.unhexlify(ans)
    ans=str(ans)
    ans=ans[2:-1]
    return(ans)

In [18]:
'''
Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers
'''


def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

In [19]:
'''
Euclid's extended algorithm for finding the multiplicative inverse of two numbers
'''


def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi

    while e > 0:
        temp1 = temp_phi//e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2

        x = x2 - temp1 * x1
        y = d - temp1 * y1

        x2 = x1
        x1 = x
        d = y1
        y1 = y

    if temp_phi == 1:
        return d + phi


In [20]:
def generate_key_pair(p, q):
    n = p * q

    # Phi is the totient of n
    phi = (p-1) * (q-1)

    # Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    # Use Euclid's Algorithm to verify that e and phi(n) are coprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    # Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)

    # Return public and private key_pair
    # Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

In [21]:
def rsa_encrypt(pk, plaintext):
    # Unpack the key into it's components
    key, n = pk
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [pow(ord(char), key, n) for char in plaintext]
    # Return the array of bytes
    return cipher

In [22]:
p = random.choice(primes)
q = random.choice(primes)
while p==q:
    q = random.choice(primes)

print(" - Generating public / private key-pairs now . . .")

public, private = generate_key_pair(p, q)

print(" - Your public key is ", public)
print(" - your private key is ", private)

 - Generating public / private key-pairs now . . .
 - Your public key is  (14257, 48359)
 - your private key is  (38521, 48359)


In [23]:
def key_generate(s_gps): 
    global key
    key = str(int(encrypted_msg)^int(s_gps))
    key=str.encode(key)
    key=binascii.hexlify(key)
    key=str(key)
    key=key[2:-1]
    key=key.zfill(32)
    key_label = Label(root, text=key, wraplength=400, font=(font_style, 10)).grid(row=6, column=1, padx=10, pady=10)

# Decrypting Cipher text


In [24]:
def Decrypt_cipher_text(c):   
    global chunkplain
    chunkplain=""
    l=[]
    while len(c)!=0:
        l.append(c[:32])
        c=c[32:]
    for i in l:
        Ciphertext=i
        chunkplain+=decrypt(Ciphertext,key)
    plain_label = Label(root, text=chunkplain, wraplength=400, font=(font_style, 14)).grid(row=11, column=1, padx=10, pady=10)

In [ ]:
root1 = Tk()
root1.title("Receiver Page")
root1.geometry("750x750")
root1.configure(bg="#f2daa9")

# Set font style
font_style = "Comic Sans MS"

# Create a canvas widget
canvas = Canvas(root1, bg="#f2daa9")
canvas.pack(side=LEFT, fill=BOTH, expand=1)

# Add a scrollbar to the canvas
scrollbar = Scrollbar(root1, orient=VERTICAL, command=canvas.yview)
scrollbar.pack(side=RIGHT, fill=Y)

# Configure the canvas to use the scrollbar
canvas.configure(yscrollcommand=scrollbar.set)
canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

# Create a frame inside the canvas for the widgets
root = Frame(canvas, bg="#f2daa9")
canvas.create_window((0,0), window=root, anchor="nw")
    

# Create labels and entry widgets for Name, Email, and Phone
lat_label = Label(root, text="Enter Your latitude ", font=(font_style, 12)).grid(row=0, column=0, padx=10, pady=10)
lat_entry = Entry(root, width=30)
lat_entry.grid(row=0, column=1, padx=10, pady=10)

lon_label = Label(root, text="Enter your longitude ", font=(font_style, 12)).grid(row=1, column=0, padx=10, pady=10)
lon_entry = Entry(root, width=30)
lon_entry.grid(row=1, column=1, padx=10, pady=10)

def encrypt_sender_gps(lat,lon):
    lat=float(lat)
    lon=float(lon)
    message =lat+lon
    global encrypted_msg
    encrypted_msg = rsa_encrypt(public, str(message))
    encrypted_msg=''.join(map(lambda x: str(x), encrypted_msg))
    
    rfile_path = r"C:\Users\krishnamurthy\Desktop\Project A3\Portal\Receiver_location.txt"
    with open(rfile_path, "w") as file:
        file.write(encrypted_msg)
    
    r_gps = Label(root, text=encrypted_msg, wraplength=400, font=(font_style, 10)).grid(row=3, column=1, padx=10, pady=10)

r_gps_button = Button(root, text="Encrypt Coordinates and share", font=(font_style, 12), bg='#543e12', relief='groove', fg='#fffdf6', bd=5, command=lambda: encrypt_sender_gps(lat_entry.get(),lon_entry.get()))
r_gps_button.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

r_gps_label = Label(root, text="Your Encrypted Coordinates :  ", font=(font_style, 12)).grid(row=3, column=0, padx=10, pady=10)

def upload_sender_gps():
    sfile_path = filedialog.askopenfilename()
    with open(sfile_path, "r") as file:
        global s_gps
        s_gps= file.read()


s_gps_button = Button(root, text="Upload Sender's Encrypted location", font=(font_style, 12), bg='#543e12', relief='groove', fg='#fffdf6', bd=5, command=lambda: upload_sender_gps())
s_gps_button.grid(row=4, column=0,columnspan=2, padx=10, pady=10)



key_label = Label(root, text="Key : ", font=(font_style, 12)).grid(row=6, column=0, padx=10, pady=10)



key_button = Button(root, text="Generate Key", font=(font_style, 12), bg='#543e12', fg='#fffdf6', relief='groove', bd=5, command=lambda: key_generate(s_gps)).grid(row=5, column=0, columnspan=2, padx=10, pady=10)


def upload_image():
    file_path = filedialog.askopenfilename()
    # Open the image file using PIL
    simg = Image.open(file_path)
    # Resize the image to fit the screen
    img = simg.resize((150, 140), Image.ANTIALIAS)
    # Convert the image to Tkinter PhotoImage format
    photo = ImageTk.PhotoImage(img)
    # Create a label to display the image
    image_label = Label(root, text="Stego Image", font=(font_style, 12)).grid(row=8, column=0, columnspan=2, padx=10, pady=10)
    image_label = Label(root, image=photo)
    image_label.image = photo
    image_label.grid(row=9, column=0,columnspan=2, padx=10, pady=10)
    global c
    os.remove(file_path)
    c = stepic.decode(simg)
    
    

upload_button = Button(root, text="Upload Image and Extract Cipher Text", font=(font_style, 12), bg='#543e12', fg='#fffdf6', bd=5, relief='groove', command=upload_image).grid(row=7, column=0, columnspan=2, padx=10, pady=10)
    
Decrypt_button = Button(root, text="Decrypt Cipher Text", font=(font_style, 12), bg='#543e12', fg='#fffdf6', relief='groove', bd=5, command=lambda: Decrypt_cipher_text(c)).grid(row=10, column=0, columnspan=2, padx=10, pady=10)


plain_label = Label(root, text="Secret text ", font=(font_style, 12)).grid(row=11, column=0, padx=10, pady=10)




root.mainloop()